## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


## 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

In [3]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [4]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [5]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [6]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [7]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4.1-nano",
    temperature=0.1,
)

## Chain 생성

### LCEL(LangChain Expression Language)

![lcel.png](./images/lcel.png)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [8]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

model = ChatOpenAI(model="gpt-4.1-nano", temperature=0.1)

chain = prompt | model

### invoke() 호출

- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.

In [9]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
input = {"topic": "인공지능 모델의 학습 원리"}

In [10]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
chain.invoke(input)

AIMessage(content="물론입니다! 인공지능 모델의 학습 원리를 쉽게 설명해 드릴게요.\n\n1. **데이터 수집**: 먼저, 인공지능이 배우기 위해 많은 예제(데이터)를 모아요. 예를 들어, 고양이 사진을 많이 보여주는 거죠.\n\n2. **모델 설계**: 그런 다음, 이 데이터를 이해하고 학습할 수 있는 '모델'이라는 수학적 구조를 만듭니다. 이 모델은 사람의 뇌처럼 작동하는 인공신경망일 수 있어요.\n\n3. **학습 과정**:\n   - 모델은 처음에는 아무것도 모르기 때문에, 입력(예: 사진)을 넣으면 결과(예: 고양이라고 예측)가 틀릴 수 있어요.\n   - 이때, 모델은 '오차'라고 하는 차이점을 계산해요. 예를 들어, 고양이 사진인데 강아지라고 말했으면 오차가 크겠죠.\n   - 그런 다음, 이 오차를 줄이기 위해 모델의 내부 값(가중치)을 조금씩 조정해요. 이 과정을 반복하면서 점점 더 정확하게 사진을 구분하게 되는 거예요.\n\n4. **반복과 개선**: 수많은 예제와 반복 학습을 통해, 모델은 점점 더 정확하게 데이터를 이해하고 예측할 수 있게 돼요.\n\n요약하자면, 인공지능 모델은 많은 데이터를 보고, 틀린 점을 수정하면서 점점 더 똑똑해지는 과정이라고 할 수 있어요!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 338, 'prompt_tokens': 22, 'total_tokens': 360, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nan

아래는 스트리밍을 출력하는 예시 입니다.

In [ ]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

물론입니다! 인공지능 모델의 학습 원리를 쉽게 설명해 드릴게요.

1. **데이터 수집**: 먼저, 인공지능이 배우기 위해 많은 예제(데이터)를 모읍니다. 예를 들어, 고양이 사진과 강아지 사진을 많이 모으는 거죠.

2. **모델 설계**: 그런 다음, 이 데이터를 이해하고 분류할 수 있도록 인공지능의 '뇌' 역할을 하는 모델을 만듭니다. 이 모델은 여러 층으로 이루어진 신경망일 수 있어요.

3. **학습 과정**:
   - **예측하기**: 모델은 처음에는 아무것도 모르기 때문에 무작위로 예측을 합니다.
   - **오차 계산**: 그리고 실제 정답(예: 사진이 고양이인지 강아지인지)과 비교해서 얼마나 틀렸는지 계산합니다.
   - **조정하기**: 틀린 정도를 줄이기 위해 모델의 내부 값(가중치)을 조금씩 수정합니다. 이 과정을 '학습'이라고 부릅니다.
   
4. **반복**: 이 과정을 수천, 수만 번 반복하면서 모델은 점점 더 정확하게 사진을 구분할 수 있게 됩니다.

5. **완성**: 충분히 학습이 되면, 새로운 사진이 들어왔을 때도 고양이인지 강아지인지 잘 맞출 수 있게 되는 거죠.

요약하자면, 인공지능은 많은 예제와 반복적인 수정 과정을 통해 점점 더 똑똑해지고, 새로운 데이터를 이해할 수 있게 되는 것입니다!

In [ ]:
from langchain_openai import ChatOpenAI

prompt = PromptTemplate.from_template("{topic}에 대해 쉽게 {how} 설명해줘.")

model = ChatOpenAI()

chain = prompt | model

In [16]:
input2 = {"topic": "인공지능 학습의 원리", "how": "3살짜리도 이해하기 쉽게"}

chain.invoke(input2)

AIMessage(content='인공지능 학습의 원리는 우리가 학습하는 것과 비슷해! 예를 들어 우리가 고양이를 배운다면, 우리는 고양이의 사진을 보고 "이게 고양이야!"라고 부모님에게 물어본다. 부모님이 맞다고 하면 우리는 그 고양이의 특징을 기억해. 인공지능도 비슷해! 인공지능이 고양이를 배울 때, 고양이의 사진을 보여주고 "이게 고양이야!"라고 말해주면, 그 특징을 기억하고 고양이를 알아볼 수 있어! 요즘엔 인공지능이 많이 발전해서 우리보다 더 빠르고 정확하게 공부할 수 있어!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 249, 'prompt_tokens': 46, 'total_tokens': 295, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C2Y2kD3bkuCrisT2IhZVEYWpsLSGf', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c2607289-5e97-4db8-ab34-969509910a41-0', usage_metadata={'input_tokens': 46, 'output_tokens': 249, 'total_tokens': 295, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_toke

In [22]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input2)
# 스트리밍 출력
stream_response(answer)

인공지능 학습은 컴퓨터가 데이터를 입력받아 패턴을 학습하여 예측이나 결정을 내리는 과정을 말합니다. 이러한 학습은 일반적으로 머신러닝 알고리즘을 사용하여 이루어지는데, 이 알고리즘은 입력된 데이터를 분석하여 패턴을 찾아내고 새로운 데이터를 분류하거나 예측하는데 사용됩니다. 이 과정에서 컴퓨터는 어떤 작업을 수행하고 그 결과를 피드백 받아 오차를 최소화하는 방향으로 학습하게 됩니다. 이러한 방식을 통해 인공지능은 점차적으로 높은 정확도와 효율성을 갖추게 됩니다.

In [23]:
from langchain_openai import ChatOpenAI

prompt = PromptTemplate.from_template("{topic}에 대해 쉽게 설명해줘.")

model = ChatOpenAI()

input = {"topic": "인공지능 학습의 원리"}

### 출력파서(Output Parser)


In [24]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Chain 에 출력파서를 추가합니다.

In [25]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

In [26]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리"}
chain.invoke(input)

'인공지능 모델의 학습 원리는 데이터를 이용하여 패턴을 학습하는 과정입니다. 먼저 모델은 입력 데이터를 받아서 내부적으로 다양한 가중치와 편향을 조절하며 출력을 예측합니다. 이때 모델이 예측한 출력과 실제 정답 간의 오차를 계산하여 이 오차를 최소화하기 위해 가중치와 편향을 조정하는 과정을 거치게 됩니다. 이렇게 하나의 데이터 샘플에 대해 오차를 최소화하는 과정을 여러 번 반복하면서 전체 데이터셋에 대한 패턴을 학습하여 모델을 훈련시킵니다. 이렇게 학습된 모델은 새로운 데이터가 들어왔을 때 정확한 출력을 예측할 수 있게 됩니다.'

In [ ]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 해볼 수 있습니다.
- `model_name` 역시 변경하여 테스트가 가능합니다.

In [27]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4.1-nano")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [28]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

In [29]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 영어 회화:
Waiter: Hello! Welcome to our restaurant. Are you ready to order?
Customer: Yes, I’d like to order the grilled chicken avec rice, please.
Waiter: Sure. Would you like anything to drink?
Customer: Just water, please. And can I get the salad on the side?
Waiter: Of course. Anything else?
Customer: No, that’s all. Thank you.

- 한글 해석:
웨이터: 안녕하세요! 저희 식당에 오신 것을 환영합니다. 주문하시겠어요?
손님: 네, 저는 치킨구이와 밥을 주문하고 싶어요.
웨이터: 알겠습니다. 음료는 어떤 걸로 하시겠어요?
손님: 물로 주세요. 그리고 샐러드도 곁들일 수 있을까요?
웨이터: 물론이죠. 더 필요하신 건 없으세요?
손님: 아니요, 그게 다예요. 감사합니다.


In [35]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:  
"Hello, I would like to order something, please."  
"Can I see the menu, please?"  
"I'll have the grilled chicken and a side salad."  
"Could I get that without onions?"  
"Thank you. How long will it take for the food to be ready?"  

- 한글 해석:  
"안녕하세요, 주문하고 싶은 게 있습니다."  
"메뉴판 좀 보여주시겠어요?"  
"그릴 치킨과 사이드 샐러드로 주세요."  
"양파 빼고 해주실 수 있나요?"  
"감사합니다. 음식이 얼마나 걸릴까요?"

In [36]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:
A: Hello! I'd like to order a large pepperoni pizza, please.  
B: Sure! Would you like anything else with that?  
A: Yes, could I also get a side of garlic bread?  
B: Absolutely. Your total comes to $20.50. Will that be for here or to go?  
A: To go, please.  
B: Great! Your order will be ready in about 15 minutes.  

- 한글 해석:
A: 안녕하세요! 큰 페퍼로니 피자 하나 주세요.  
B: 네! 그것 외에 더 주문하실 거 있으신가요?  
A: 네, 마늘 빵도 하나 더 주세요.  
B: 알겠습니다. 총 금액은 20.50달러입니다. 여기서 드실 건가요, 포장하시겠어요?  
A: 포장해 주세요.  
B: 알겠습니다! 주문하신 음식은 약 15분 후에 준비됩니다.